**Gist:**

* There are 23023 articles in the data set with an average length of 1066 tokens
* from these articles, 624 contain the words "AI", "A.I.", "Artificial Intelligence", "Machine Learning", "Deep Learning" or "Neural Network"
* The average length of the articles in the subset is 1749 tokens

In [2]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(stringr)

In [3]:
articles <- read.csv('dsjVoxArticles.tsv', sep="\t", row.names=NULL)

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“EOF in Zeichenkette”

In [4]:
head(articles)

title,author,category,published_date,updated_on,slug,blurb,body


In [5]:
summary(articles)

                                                                            title      
 Caroline Framke                                                               :    6  
 Democratic debate 2016 live stream: time, TV schedule, and how to watch online:    4  
 Tara Golshan                                                                  :    4  
 Andrew Prokop                                                                 :    3  
 Jeff Stein                                                                    :    3  
 Republican debate 2016 live stream: time, TV schedule, how to watch online    :    3  
 (Other)                                                                       :10594  
              author                  category                published_date 
 German Lopez    :1022   The Latest       :3255                      :   37  
 Matthew Yglesias: 716   Donald Trump     : 764   2016-07-13 12:00:03:    8  
 Dara Lind       : 554   Politics & Policy: 633   2017-01-28 2

In [6]:
nrow(articles)

[1] 10617

The summary shows that the articles in the data frames still contain html-tags, so I scrape them first. While scraping them, a few rows threw type errors. I catched the errors, and saved the texts that threw type errors in a separate tsv-file.

First, I look at the files that could be scraped succesfully:

In [7]:
articles_scraped <- read.csv('dsjVoxArticles_scraped.tsv', sep="\t", row.names=NULL)

In [8]:
nrow(articles_scraped)

[1] 23024

In [9]:
error_articles <- read.csv('errors_dsjVoxArticles_scraped.tsv', sep="\t", row.names=NULL)

In [10]:
nrow(error_articles)

[1] 23024

There are two interesting observations:

1. The dataframe with erraneous files contains as many rows than the dataframe without the erraneous files

2. The dataframe without preprocessing contains less rows than the preprocessed dataframes, and throws an error that says "EOF in String"

I checked the downloaded .tsv file, and it contains indeed 23023 rows, as much as the preprocessed files. Therefore, the original frame contained a string that broke the loading process, and this string was filtered out during scraping. So this problem is solved, and it does not make sense to investigate it further.

It also seems that each file threw a type error while scraping, hence the doubling of the texts in the data frame. The type error was something about float and integer. I don't think that this error affects the work later, so I just use the file with the scraped articles for the data analysis.

In [8]:
summary(articles_scraped)

       X        
 Min.   :    0  
 1st Qu.: 5756  
 Median :11512  
 Mean   :11512  
 3rd Qu.:17267  
 Max.   :23023  
                
                                                                            title      
 Republican debate 2016 live stream: time, TV schedule, how to watch online    :    7  
 Republican debate 2016: start time, schedule, and what to expect              :    6  
 Democratic debate 2016 live stream: time, TV schedule, and how to watch online:    5  
 Democratic debate 2016: start time, schedule, and what to expect              :    4  
 Democratic debate 2015 live stream: time, TV schedule, and how to watch online:    3  
 Democratic debate 2015: start time, schedule, and what to expect              :    3  
 (Other)                                                                       :22996  
              author                   category                 published_date 
 German Lopez    : 2243   The Latest       : 7152                      :   11  


The summary shows that all the html-tags are gone (as intended), but the substring '\\n' occurs often in the article. This is a substring that can be filtered out later, probably in the tokenization step.

Now I calculate the average length of the articles:

In [9]:
articles_scraped['length'] <- str_count(articles_scraped$body, pattern = " ")

In [12]:
summary(articles_scraped)

       X        
 Min.   :    0  
 1st Qu.: 5756  
 Median :11512  
 Mean   :11512  
 3rd Qu.:17267  
 Max.   :23023  
                
                                                                            title      
 Republican debate 2016 live stream: time, TV schedule, how to watch online    :    7  
 Republican debate 2016: start time, schedule, and what to expect              :    6  
 Democratic debate 2016 live stream: time, TV schedule, and how to watch online:    5  
 Democratic debate 2016: start time, schedule, and what to expect              :    4  
 Democratic debate 2015 live stream: time, TV schedule, and how to watch online:    3  
 Democratic debate 2015: start time, schedule, and what to expect              :    3  
 (Other)                                                                       :22996  
              author                   category                 published_date 
 German Lopez    : 2243   The Latest       : 7152                      :   11  


The average length of the articles is 1066 tokens.

The next step is to find out how many of these texts contain the words "Artificial Intelligence" or "AI". For this, I will search for matches with grepl() and regex.

In [13]:
ai_data <- select(articles_scraped, everything())%>%
    filter(str_detect(articles_scraped$body,'AI'))

artificial_int_data <- select(articles_scraped, everything())%>%
    filter(str_detect(articles_scraped$body,'artificial intelligence'))

a_i_data <- select(articles_scraped, everything())%>%
    filter(str_detect(articles_scraped$body,'A\\.I\\.'))

art_int_upper <- select(articles_scraped, everything())%>%
    filter(str_detect(articles_scraped$body,'Artificial Intelligence'))


In [14]:
nrow(ai_data)

[1] 559

In [15]:
nrow(artificial_int_data)

[1] 78

In [16]:
nrow(a_i_data)

[1] 6

In [17]:
nrow(art_int_upper)

[1] 3

In [18]:
ai_all <- bind_rows(ai_data, artificial_int_data, a_i_data, art_int_upper)
    

In [19]:
nrow(ai_all)

[1] 646

In [20]:
ai_all = distinct(ai_all)

nrow(ai_all)

[1] 610

610 articles are somehow related to Artificial Intelligence. To broaden the corpus a bit, I will look up if there are any articles that contain the words "Machine Learning", "Neural Networks", and "Deep Learning".

In [21]:
ml_data <- select(articles_scraped, everything())%>%
    filter(str_detect(articles_scraped$body,regex('machine learning', ignore_case = T)))

nrow(ml_data)

[1] 23

In [22]:
nn_data <- select(articles_scraped, everything())%>%
    filter(str_detect(articles_scraped$body,regex('neural network', ignore_case = T)))

nrow(nn_data)

[1] 10

In [23]:
deep_learning <- select(articles_scraped, everything())%>%
    filter(str_detect(articles_scraped$body,regex('deep learning', ignore_case = T)))

nrow(deep_learning)

[1] 4

In [24]:
ai_all = bind_rows(ai_all, ml_data, nn_data, deep_learning)

ai_all = distinct(ai_all)

nrow(ai_all)

[1] 624

We now have sampled 624 articles that are somehow related to machine learning/AI. Some of them might not be useful for us, but most of them should be. This are also enough articles to annotate 120 of them manually.


In [25]:
summary(ai_all)

       X        
 Min.   :    1  
 1st Qu.: 5546  
 Median :11648  
 Mean   :11440  
 3rd Qu.:17354  
 Max.   :23012  
                
                                                                           title    
 Here's a list of aid groups working on the Ebola crisis â€” and how to donate:  2  
  Descender is the best new comic book of 2015                                :  1  
  The man who discovered Ebola on why this epidemic spiraled out of control   :  1  
 #Gamergate: Here's why everybody in the video game world is fighting         :  1  
 #NeverTrump and the coming schism in the Republican Party, explained         :  1  
 #WheresRey and the big Star Wars toy controversy, explained                  :  1  
 (Other)                                                                      :617  
              author                  category               published_date
 German Lopez    : 66   The Latest        :204   2015-07-09 11:00:02:  2   
 Julia Belluz    : 56   Politics

The mean length of the articles in this subset is longer than in the overall dataset. The length of articles in the full dataset is 1066 tokens, in this subset, the average length is 1749 tokens.

In [28]:
write.csv(ai_all, 'AI_subset.csv')